# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup, Tag, NavigableString
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
jeopardy = pd.read_csv("jeopardy.csv")
#rename the columns with useable names - remove whitespace, use lower case, change round and value to non-reserved names
jeopardy = jeopardy.rename(columns = {'Show Number':'show_number', ' Air Date':'air_date', ' Round':'game_round', ' Category':'category', ' Value': 'dollar_value',' Question':'question', ' Answer':'answer'})

In [2]:
#Check for duplicate questions & filter to unique questions
duplicates = jeopardy.duplicated().any()
#print(duplicates)
#no full rows are duplicated, but individual questions are:
duplicate_questions = jeopardy.question.duplicated().any()
#print(duplicate_questions)
#Remove lines containing filler questions,missing audio/video clues, There are several questions that contain [video clue], [audio clue], or [filler], remove those
odd_questions = jeopardy[jeopardy.question.str.contains('\[')]
bad_questions = ['[audio clue]', '[video clue]', '[filler]']
filter_questions = jeopardy[jeopardy.question.isin(bad_questions)]
jeopardy = pd.concat([jeopardy, filter_questions]).drop_duplicates(keep = False)
# Drop any remaining duplicate questions
jeopardy = jeopardy.drop_duplicates('question')
duplicate_questions = jeopardy.question.duplicated().any()
#print(duplicate_questions)


In [3]:
#print(jeopardy.count())
#There are still two fewer answers than questions.
#print(jeopardy[jeopardy.answer.isnull()])
#Funny. The answer to the question is 'null' so it got pulled in as a Null value. Let's see if we can fix that.
jeopardy.at[94817, 'answer'] = 'null'
jeopardy.at[143297, 'answer'] = 'null'
print(jeopardy.count())

show_number     216121
air_date        216121
game_round      216121
category        216121
dollar_value    216121
question        216121
answer          216121
dtype: int64


In [4]:
#Remove punctuation from game rounds
print(jeopardy.game_round.unique())
jeopardy['game_round'] = jeopardy.game_round.apply(lambda x: x.replace('!', ''))
print(jeopardy.game_round.unique())

['Jeopardy!' 'Double Jeopardy!' 'Final Jeopardy!' 'Tiebreaker']
['Jeopardy' 'Double Jeopardy' 'Final Jeopardy' 'Tiebreaker']


In [5]:
#Categories probably need some clean-up
print(jeopardy.category.count())
print(jeopardy.category.nunique())
jeopardy['category'] = jeopardy.category.apply(lambda x: x.strip())
#replace the category that's just an ! mark with the words 'exclamation mark [altered]'
jeopardy['category_words']= jeopardy['category']
jeopardy['category_words'] = jeopardy.category.apply(lambda x: x.lower())
exclaim_ind = jeopardy.index[jeopardy.category_words =='"!"'].to_list()
for idx in exclaim_ind:
    jeopardy.at[idx, 'category_words'] = 'exclamation mark [altered]'
#Remove the special characters and just let the words exist in a new column
def replace_in_column (column, **kwargs):
    for key, value in kwargs.items():
        altered_col = column.str.replace(key, value, regex = True)
    return altered_col
replace_punctuation = {'&': 'and','-': ' ', '_*': '(blank)', '[^\w\s]':''}
jeopardy['category_words'] = replace_in_column(jeopardy['category_words'],**replace_punctuation)


216121
27992


In [41]:
#Clean up questions in the same way
#create a link column from the links in text string in a pandas column
def link_finder (column):
       soup = BeautifulSoup(column, 'html.parser')
       links = soup.findAll('a', href = True)
       urls = []
       link_texts = []
       if links != []:
              for link in links:
                     url = link['href']
                     urls.append(url)
       return urls
jeopardy['links'] = jeopardy.question.apply(link_finder)
print(jeopardy.links.head(50))

0                                                        []
1                                                        []
2                                                        []
3                                                        []
4                                                        []
5                                                        []
6                                                        []
7                                                        []
8                                                        []
9                                                        []
10                                                       []
11                                                       []
12                                                       []
13                                                       []
14                                                       []
15                                                       []
16                                      

In [44]:
#Replace the a href in the question with the link text. 
def link_remover (column):
       soup = BeautifulSoup(column, 'html.parser')
       links = soup.findAll('a', href = True)
       urls = []
       link_texts = []
       if links != []:
              for link in links:
                     link_text=link.text
                     column= column.replace(str(link), link_text)
       return column
jeopardy['question_no_link'] = jeopardy.question.apply(link_remover)
print(jeopardy.question_no_link.head(50))

0                                                                                       For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory
1                                                                            No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves
2                                                                                               The city of Yuma in this state has a record average of 4,055 hours of sunshine each year
3                                                                                                   In 1963, live on "The Art Linkletter Show", this company served its billionth burger
4                                                                               Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States
5                                                                          

Interesting note. On trying to remove the whitespace from the individual rows of the questions, I learned that the whitespace exists because of the variable lengths of strings. Pandas needs to keep each row the same space so assigns strings as objects in the dataframe and adds white space. As I need to do things to the strings I can transform them from objects to strings.

In [ ]:
def find_url (column):
    column_txt = str(column )

In [ ]:
print(jeopardy.question.apply(link_finder).head(50))

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [ ]:
#function to find words from a words list in a dataframe column, count them, and return dictionaris of the word:count, and the word:column-rows. 
def words_list_counter (column, words_list):
    wordlist_questions = {}
    word_list_count = {}
    for word in words_list:
        count = column[column.str.contains(word)].count()
        word_questions = column[column.str.contains(word)].reset_index(drop=True)
        word_list_count[word]=count
        wordlist_questions [word] = word_questions
    return word_list_count, wordlist_questions


In [ ]:
#find the questions that include a US state
state_list = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
states_questions_count, states_questions = words_list_counter(jeopardy['question'], state_list)
print (states_questions_count)
print(states_questions)

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [ ]:
#test the function to find questions about topics with terrains.
terrain_list = ["land", "sea", "ocean", "mountain", "forest", "plains", "grassland", "savannah", "tundra", "canyon", "dessert", "marsh", "swamp"]
terrain_questions_count, terrain_questions = words_list_counter(jeopardy['question'], terrain_list)
print(terrain_questions_count)
print(terrain_questions)

4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.